# DLMI Lecture 03

In this assignment you will implement methods for standardizing the orientation of a medical image.

In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np
import time 
import sys
import os 

In [ ]:
engr_dir = "/opt/nfsopt/DLMI"
idas_dir = os.path.join(os.path.expanduser('~'), "classdata")

if os.path.isdir(engr_dir):
    data_dir = engr_dir
elif os.path.isdir(idas_dir):  
    data_dir = idas_dir
else:
    print("Data directory not found")

### How to display image in Python

If the direction cosine matrix is identity, the $ijk$ image coordinate system basis has the same orientation as the $lps$ anatomical coordinate system basis. Therefore:

- $i$ : right -> $l$eft axis
- $j$ : anterior -> $p$osterior axis
- $k$ : inferior -> $s$uperior axis

Therefore the anatomical planes are accessed in the following way given SimpleITK image object `im`:

- `im[i, :, :]` -> sagittal slice $i$
- `im[:, j, :]` -> coronal slice $j$
- `im[:, :, k]` -> axial slice $k$

This is **ONLY** true if the direction cosine matrix is identity.

When a SimpleITK image is converted to a NumPy matrix, the first and last dimensions get permuted. Therefore voxel `[i, j, k]` in a SimpleITK image corresponds to NumPy matrix index `[k, j, i]`. Similarly, accessing the anatomical planes are permuted. Given a NumPy matrix `imnp`:

- `im_np[:, :, i]` -> sagittal slice $i$
- `im_np[:, j, :]` -> coronal slice $j$
- `im_np[k, :, :]` -> axial slice $k$

Again, this is only true if the direction cosine matrix is identity. 

Aside: When displaying a NumPy matrix with matplotlib, the `imshow` origin (where the [0,0] index of the array is placed when displaying an image) is by default the top left corner. The matrix will be displayed with the row index (first dimension) increasing downwards and the column index (second dimension) increasing to the right. Although this is not the same convention used for images, everything works out nicely since the first and last dimension of the image were permuted when converting to NumPy. As a result, all the anatomical planes get displayed in the correct orientation. The only exception is the superior/inferior axis. Given an identity direction cosine matrix, the `k` index will increase in the superior direction. Since the `imshow` origin is the top left corner, this will result in the anatomy being displayed upside down in the coronal and sagittal views. In other words, the superior direction is down when it should be up. A simple fix is to set `origin = lower` when displaying coronal and sagittal slices.

### Standardize image orientation

In this assignment you will work with an image that has a non-identity direction cosine matrix, and you will standardize the image such that it has an identity cosine matrix. This will ensure all images are oriented the same and that we can use the above conventions to consistently access / display the anatomical planes.

First load the following two images in Slicer:
- `CTChest_scan1.nii.gz`
- `CTChest_scan2.nii.gz`.

Both images are sampled from the same underlying function. However, the image coordinate systems defintion with respect to the anatomical coordinate system is different. 

Do the images appear the same in Slicer? (answer below).

XXX

Now read the images using SimpleITK.

In [ ]:
fn1 = os.path.join(data_dir, "lecture03", "CTChest_scan1.nii.gz")
fn2 = os.path.join(data_dir, "lecture03", "CTChest_scan2.nii.gz")

im1 = sitk.ReadImage(fn1)
im2 = sitk.ReadImage(fn2)

im1np = sitk.GetArrayFromImage(im1)
im2np = sitk.GetArrayFromImage(im2)

### Part 1

Complete the function below for displaying a figure with 3 subplots. The subplots should display the axial, coronal, and sagittal views of a 3D image. Your code can assume that the input image will have an identity direction matrix. For credit, your code will need to properly display input images that have non-isotropic voxels. Additionally, the orientation needs to be correctly displayed (should match the orientation in Slicer). Hint: you may need to change the imshow origin for some views, see aside above. Complete the function below by replacing the XXXs with appropriate code.

In [ ]:
def myshow_multiview(im, slice_inds, min_intensity, max_intensity):
    """
    Displays three inline images corresponding to the anatomical planes froma 3D image.

    The input image must be 3D.

    Assumes an identity direction cosine matrix!

    Inputs:
    - im (sitk.Image): SimpleITK image to display
    - slice_inds (int, int, int): Tuple with indices of slices to display: (sagittal_ind, coronal_ind, axial_ind)
    - min_intensity (int): Minimum display intensity (mapped to black)
    - max_intensity (int): Maximum display intensity (mapped to white)

    Returns:
    """
    
    # Create figure with 3 subplots    
    fig, ax = plt.subplots(1,3, figsize = (12,4))

    # Convert SimpleITK image to Numpy Array
    XXX
    
    # Display each slice
    ax[0].imshow(XXX) # Sagittal slice
    ax[1].imshow(XXX) # Coronal slice
    ax[2].imshow(XXX) # Axial slice

    # Add title to each view
    planes = ["Sagittal", "Coronal", "Axial"]
    for i,p in enumerate(planes):
        ax[i].set_title(p)

Below, call the `myshow_multiview` function to display `im1`. `im1` is provided as a reference image to check your `myshow_multiview` implementation, as it follows the assumption of having an identify direction cosine matrix. For full credit, the displayed image must match what you obsereve when opening `im1` in Slicer (i.e., aspect ratio and orientation for each view).

In [ ]:
######################################
#######         TODO           #######
######################################

Naively, try calling the `myshow_multiview` function to display `im2`.

In [ ]:
######################################
#######         TODO           #######
######################################

Compare the three views of `im1` and `im2`. How are they the same or different?

XXX

Next, compare the meta data (size, spacing, origin, direction cosine matrix) betewen `im1` and `im2`.
Describe the similarities/differences (in the markdown cell below).

In [ ]:
######################################
#######         TODO           #######
######################################

XXX

Note: the `GetDirection()` class method, returns a tuple of 9 elements: a vector representing a matrix in row major order. To get the corresponding $3\times3$ matrix you can convert it to a numpy array, and use the `np.reshape()` function, i.e., `np.array( im.GetDirection() ).reshape(3,3)` given SimpleITK image `im`.

In the remainder of the assignment you wil implement the two approaches for standardizing an image such that the direction cosine matrix is transformed to the identity matrix. 


### Part 2

Implement the more general approach for orientation standardization using `sitk.ResampleImageFilter` as described in Lecture 03. In this approach you should also standardize the spacing to (1,1,1). Apply the method to `im2`. The function should only have a single input - the image to be standardized (`im1` should not be used in the remainder of the assignment).

In [ ]:
def standardize_resample(im):
    """
    Standardizes the orientation (identity) and spacing (1mm isotropic) 
    of an input image using the resampling approach.

    Inputs:
    - im (sitk.Image): SimpleITK image to standardize

    Returns:
    - im_stand (sitk.Image): SimpleITK image standardized image
    """
    ######################################
    #######         TODO           #######
    ######################################

    

    return im_stand


# call function with im2 as input
im2_stand_a = standardize_resample(im2)

Display the 3 views of the standardized `im2` to confirm the expected orientation. Also, print the metadata for the standardized image.

In [ ]:
######################################
#######         TODO           #######
######################################

### Part 3

Implement the standardization approach which assumes the direction cosine matrix consists of 0 and $\pm1$. The identity matrix can be achieved by combinations of permutations and/or flips of the (i j k) coordinate system. SimpleITK has a `sitk.PermuteAxes()` and `sitk.Flip()` filter. This can also be accomplished using NumPy functions `np.transpose()` and `np.flip()`, but be sure to consider the NumPy indices are `[k, j, i]`.  

In [ ]:
######################################
#######         TODO           #######
######################################

def standardize_permute_flip(im):
    """
    Standardizes the orientation (identity) 
    of an input image using the permute and flip approach.

    Inputs:
    - im (sitk.Image): SimpleITK image to standardize

    Returns:
    - im_stand (sitk.Image): SimpleITK image standardized image
    """
    ######################################
    #######         TODO           #######
    ######################################

    

    return im_stand


# call function with im2 as input
im2_stand_b = standardize_permute_flip(im2)

Display the 3 views of the standardized image to confirm the expected orientation. Also, print the metadata for the standardized image.

In [ ]:
######################################
#######         TODO           #######
######################################

How do the two methods compare? Do they produce the same or different standardized image? How does the metadata compare to the original image?

XXX